## The Typing Map
![typingMap](./Images/typingMap.png)

## Two kinds of protocols
An object protocol specifies methods which an object must provide to fulfill a role. 
- dynamic protocol: Dynamic protocols are implicit, defined by convention and described in the documentation. (e.g. a sequence protocol need `__getitem__` and `__len__`, but only `__getitem__` is already sufficient in some context)
- static protocol: A static protocol has an explicit definition with `typing.Protocol` to define one or more methods that a class must implement (or inherit) to satisfy a static type checker.

Difference:
- An object may implement only part of a dynamic protocol and still be useful; but to fulfill a static protocol, the object must provide every method declared in the protocol class, even if your program doesn’t need them all.

- Static protocols can be verified by static type checkers, but dynamic protocols can’t.


## Programming ducks
### Python digs Sequences
Given the importance of sequence-like data structures, Python manages to make iteration and the in operator work by invoking `__getitem__` when `__iter__` and `__contains__` are unavailable.

In [1]:
import collections
Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list("JQKA")
    suits = "spades diamonds clubs hearts".split()
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
    
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]
    
deck = FrenchDeck() # `deck` is a sequence-like object

In [2]:
from random import shuffle
l = list(range(10))
shuffle(l)
print(l)
try:
    shuffle(deck)
except TypeError as e:
    print(e) # Need to invoke `__setitem__`, but not available

[3, 7, 1, 5, 2, 0, 6, 8, 4, 9]
'FrenchDeck' object does not support item assignment


### Implementing a Protocol at Runtime
What if we can't change the definition of `FrenchDeck` now? We can implement this method at runtime: dynamic protocol is NOT verified by static type checker

In [3]:
def set_card(deck, position, card):
    deck._cards[position] = card

FrenchDeck.__setitem__ = set_card
shuffle(deck)
deck[:5]

[Card(rank='2', suit='clubs'),
 Card(rank='J', suit='spades'),
 Card(rank='10', suit='spades'),
 Card(rank='2', suit='spades'),
 Card(rank='5', suit='clubs')]

### Defensive programming and 'fail fast'
Failing fast means raising runtime errors as soon as possible, for example, rejecting invalid arguments right at the beginning of a function body.

## Goose typing
An abstract class represents an interface.

goose typing entails:

- Subclassing from ABCs to make it explicit that you are implementing a previously defined interface.

- Runtime type checking using ABCs instead of concrete classes as the second argument for isinstance and issubclass.

Such as implementing `FrenchDeck2` as a subclass of `MutableSequence`, we have to implement `__delitem__` and `insert`. But we can inherit five concrete methods from `Sequence`: `__contains__`, `__iter__`, `__reversed__`, `index`, `count` and another six methods from `MutableSequence`: `append`, `reverse`, `extend`, `pop`, `remove` and `__iadd__`(`+=`)

In [4]:
import collections
from random import shuffle

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(collections.abc.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = "spades diamonds clubs hearts".split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
    def __len__(self):
        return len(self._cards)
    def __getitem__(self, position):
        return self._cards[position]
    def __setitem__(self, position, value):
        self._cards[position] = value
    # subclassing MutableSequence forces us to implement `__delitem__`, an abstract method of that ABC.
    def __delitem__(self, position): 
        del self._cards[position]
    # subclassing MutableSequence forces us to implement `insert`, an abstract method of that ABC.
    def insert(self, position, value):
        self._cards.insert(position, value)

deck = FrenchDeck2()
shuffle(deck)
print(deck[:5])

[Card(rank='4', suit='clubs'), Card(rank='7', suit='clubs'), Card(rank='9', suit='clubs'), Card(rank='8', suit='diamonds'), Card(rank='5', suit='hearts')]


## ABCs in the Standard Library
Most widely used ABCs are defined in `collections.abc`. But if we want to create a brand new ABC, we need to inherit from `abc.ABC` (seperated from package `collections`)
![ABCs](./Images/ABCs.png)
- `Iterable`, `Container`, `Sized`

    Every collection should either inherit from these ABCs or implement compatible protocols. `Iterable` supports iteration with `__iter__`, `Container` supports the `in` operator with `__contains__`, and `Sized` supports `len()` with `__len__`.

- `Collection`

    This ABC has no methods of its own, but was added in Python 3.6 to make it easier to subclass from `Iterable`, `Container`, and `Sized`.
- `Sequence`, `Mapping`, `Set`

    These are the main immutable collection types, and each has a mutable subclass.
    
    ![MutableSequence](./Images/MutableSequence.png)
    ![MutableMapping](./Images/MutableMapping.png)
    ![MutableSet](./Images/MutableSet.png)
- `MappingView`

In Python 3, the objects returned from the mapping methods `.items()`, `.keys()`, and `.values()` implement the interfaces defined in `ItemsView`, `KeysView`, and `ValuesView`, respectively. 

## Defining and Using an ABC
Now let’s assume we are building an ad management framework called `ADAM`. One of its requirements is to support user-provided non-repeating random-picking classes. To make it clear to ADAM users what is expected of a “non-repeating random-picking” component, we’ll define an ABC called `Tombola`
![Tombola](./Images/Tombola.png)

An abstract method can actually have an implementation. Even if it does, subclasses will still be forced to override it, but they will be able to invoke the abstract method with `super()`, adding functionality to it instead of implementing from scratch. 

In [5]:
import abc

# To define an ABC, subclass `abc.ABC`
class Tombola(abc.ABC):
    # `abstractmethod` in Python is like pure virtual function in CPP, it's often empty except for a docstring
    @abc.abstractmethod
    def load(self, iterable):
        """Add items from an iterable"""

    @abc.abstractmethod
    def pick(self):
        """Remove item at random, returning it. This method should raise `LookupError` when the instance is empty."""

    def loaded(self):
        """Return `True` if there's at least 1 item, `False` otherwise."""
        return bool(self.inspect())
    # easy but very expensive, but correct for all subclasses, could be overrided by subclasses
    def inspect(self):
        """Return a sorted tuple with the items currently inside."""
        items = []
        # pop all items into a list `items`
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        # push everything back
        self.load(items)
        return tuple(items)

In [6]:
import random

# still use the silly method from Tombola
class BingoCage(Tombola):
    def __init__(self, items):
        self._randomzier = random.SystemRandom()
        self._items = []
        self.load(items)

    def load(self, items):
        self._items.extend(items)
        self._randomzier.shuffle(self._items)
    
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    def __call__(self):
        self.pick()

class LottoBlower(Tombola):
    def __init__(seld, iterable):
        self._balls = list(iterable)
    def load(self, iterable):
        self._balls.extend(iterable)
    def pick(self):
        try:
            position = random,randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty LottoBlower')
        return self._balls.pop(position)
    def loaded(self):
        return bool(self._balls)
    def inspect(self):
        return tuple(self._balls)

### A virtual subclass of `Tombola`
We can register a class as a *virtual subclass* of an ABC, even if it does not inherit from it. When doing so, we promise that the class faithfully implements the interface defined in the ABC—and Python will believe us without checking.

This is done by calling a `register` class method on the ABC. The registered class then becomes a virtual subclass of the ABC, and will be recognized as such by `issubclass` and `isinstance`, but it does not inherit any methods or attributes from the ABC.

Virtual subclass `TomboList` works as follows:

![TomboList](./Images/TomboList.png)

In [7]:
from random import randrange

@Tombola.register
class TomoboList(list):
    # TomoboList inherit `__init__`, '__len__`, `__bool__` und `extend` from list 
    load = list.extend

    # self is itself a list
    def pick(self): 
        if self:
            position = randrange(len(self))
            return self.pop(position)
        else:
            raise LookupError('pop from empty TomboList')
    def loaded(self):
        return bool(self) 

    def inspect(self):
        return tuple(self)
# Tombola.register(TomboList) # alternative for decorator

# just like `list(range(100))`
t = TomoboList(range(100))
issubclass(TomoboList, Tombola), isinstance(t, Tombola)

(True, True)

Inheritance is guided by a special class attribute named `__mro__`(Method Resolution Order). It lists only the “real” superclasses

In [8]:
TomoboList.__mro__

(__main__.TomoboList, list, object)